In [3]:
from bs4 import BeautifulSoup
import os
import shutil
def process_html_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            soup = BeautifulSoup(content, 'html.parser')
            
            # Tìm và xử lý các tiêu đề
            titles = []
            for tag in ['h2', 'h4', 'h6']:
                headers = soup.find_all(tag)
                for header in headers:
                    titles.append(header.get_text().strip())
            
            # Tìm div có id="chapter-content"
            chapter_content = soup.find('div', id='chapter-content')
            
            if chapter_content:
                # Lấy tất cả các thẻ p và giữ nguyên định dạng
                paragraphs = []
                for p in chapter_content.find_all('p'):
                    # Xử lý các thẻ định dạng đặc biệt
                    formatted_text = ''
                    for element in p.contents:
                        if element.name == 'b':
                            formatted_text += f'**{element.text}**'
                        elif element.name == 'i':
                            formatted_text += f'*{element.text}*'
                        elif element.name == 'br':
                            formatted_text += '\n'
                        else:
                            formatted_text += str(element)
                    
                    paragraphs.append(formatted_text.strip())
                
                # Kết hợp tiêu đề và nội dung
                final_content = '\n\n'.join(titles) + '\n\n' + '\n\n'.join(paragraphs)
                return final_content
            return None
    except Exception as e:
        print(f"Lỗi khi xử lý file {file_path}: {str(e)}")
        return None

def create_mirror_structure(src_dir, dest_dir):
    """Tạo cấu trúc thư mục giống hệt từ thư mục nguồn sang thư mục đích"""
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

def process_directory(src_dir, dest_dir):
    # Tạo thư mục đích nếu chưa tồn tại
    create_mirror_structure(src_dir, dest_dir)
    
    # Duyệt qua tất cả các mục trong thư mục nguồn
    for item in os.listdir(src_dir):
        src_path = os.path.join(src_dir, item)
        dest_path = os.path.join(dest_dir, item)
        
        if os.path.isdir(src_path):
            # Nếu là thư mục, đệ quy để xử lý
            process_directory(src_path, dest_path)
        elif item.endswith('.html'):
            # Nếu là file html, xử lý và lưu kết quả
            content = process_html_file(src_path)
            if content:
                # Đổi đuôi file từ .html sang .txt
                dest_path = dest_path.replace('.html', '.txt')
                
                # Tạo thư mục cha nếu chưa tồn tại
                os.makedirs(os.path.dirname(dest_path), exist_ok=True)
                
                # Lưu nội dung đã xử lý
                with open(dest_path, 'w', encoding='utf-8') as f:
                    f.write(content)
                print(f"Đã xử lý: {src_path} -> {dest_path}")

if __name__ == "__main__":
    # Thư mục gốc chứa truyện
    source_directory = "Truyen_done"
    
    # Thư mục đích để lưu kết quả
    destination_directory = "Truyen_extracted"
    
    # Xử lý toàn bộ cấu trúc thư mục
    process_directory(source_directory, destination_directory)